# Imviz Demonstration Notebook
## Roman/ASDF Edition

This notebook demonstrates the Imviz API in the Notebook setting when using simulated observations from The Nancy Grace Roman Space Telescope. UI equivalents for these actions, as well as additional documentation about Imviz, can be found here: https://jdaviz.readthedocs.io/en/latest/imviz/

Import modules needed for this notebook.

In [ ]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Table
from regions import PixCoord, CirclePixelRegion, CircleSkyRegion

from jdaviz import Imviz

We also need this to display Matplotlib in the notebook later.

In [ ]:
%matplotlib inline

For now, we will use synthetic data for testing.

In [ ]:
from jdaviz.configs.imviz.tests.utils import create_wfi_image_model

image_model = create_wfi_image_model((20, 10))

In [ ]:
# load the observations into the helper
imviz = Imviz()

with warnings.catch_warnings():
    warnings.simplefilter('ignore')  # ErfaWarning
    imviz.load_data(image_model, ext='data')

Then, we visualize the data and start off by looking at some of the basic features:

In [ ]:
imviz.show()

Imviz will initially display Roman images zoomed to fit the display. However, since each detector's images have 4k resolution, the default Imviz zoom makes it hard to tell if the colormap stretch is appropriate.

Panning and zooming is possible by showing the viewer toolbar and clicking on the '+'-shaped icon, then dragging around in the image and using scrolling to zoom in and out. To change the stretch and colormap, show the **Layer** options accessible through the last icon in the viewer toolbar.

We can also change these programmatically, for example the stretch:

In [ ]:
for viewer_name in imviz.app.get_viewer_reference_names():
    viewer = imviz.app.get_viewer(viewer_name)

    # set colormap and stretch:
    viewer.stretch = 'log'
    viewer.set_colormap('Viridis')
    viewer.cuts = [1, 1e4]
    
    # set zoom level
    viewer.zoom(1.25)

Note also that in the above example there are mouse-over coordinates visible by default.

It possible to make selections/regions in images and export these to Astropy regions. Click on the viewer toolbar then click on the circular selection tool, and drag and click to select an interesting region on the sky. We can then export this region with:

In [ ]:
regions = imviz.get_interactive_regions()

In [ ]:
regions

Since the region is an Astropy region, we can e.g. convert it to a mask:

In [ ]:
mask = regions['Subset 1'].to_mask(mode='subpixels')

In [ ]:
data = imviz.get_data(imviz.app.data_collection[0].label)

In [ ]:
plt.imshow(mask.to_image(data.data.shape), origin='lower')

You can use Astrowidgets API.

In [ ]:
# Center the image on given pixel position.
viewer.center_on((6, 5))  # X, Y (0-indexed)

In [ ]:
# Move the image with the given pixel offsets.
viewer.offset_by(5, -3)  # dX, dY

We can convert the coordinates of the interactive subset from pixel to world coordinates like so:

In [ ]:
center = regions['Subset 1'].center
data.wcs.pixel_to_world([center.x], [center.y]).to_string('hmsdms')

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord(ra="13h11m34.25069521s", dec="-01d21m55.92127995s")
viewer.center_on(sky)

In [ ]:
# Move the image with the given sky offsets.
viewer.offset_by(0.5 * u.arcsec, -1.5 * u.arcsec)

You can programmatically zoom in and out.

Zoom level:

* 1 means real-pixel-size.
* 2 means zoomed in by a factor of 2.
* 0.5 means zoomed out by a factor of 2.
* 'fit' means zoomed to fit the whole image width into display.

In [ ]:
# Get the current zoom level.
viewer.zoom_level

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 1

In [ ]:
# Set the relative zoom based on current zoom level.
viewer.zoom(2)

It is also possible to programmatically add non-interactive markers to the image.

In [ ]:
# Add 20 randomly generated X,Y (0-indexed) w.r.t. reference image
# using default marker properties.
image_shape = data.data.shape
regs_xy = [CirclePixelRegion(PixCoord(x, y), 3)
           for x, y in zip(np.random.randint(0, image_shape[1], 20),
                           np.random.randint(0, image_shape[0], 20))]
viewer.add_markers(regs_xy)

You could customize marker color, alpha, size, and fill with values that Glue supports.

In [ ]:
viewer.marker = {'color': 'green', 'alpha': 0.8, 'fill': False}

In [ ]:
np.random.seed(42)
sky_radius = 0.1 * u.arcsec
# Mark some sky coordinates using updated marker properties.
regs_sky = [CircleSkyRegion(sky.spherical_offsets_by(*np.random.normal(scale=0.1, size=2) * u.arcsec), sky_radius) for _ in range(5)]
viewer.add_markers(regs_sky, marker_name='my_sky')

When you do not need the markers anymore, you could remove them by name.

In [ ]:
viewer.remove_markers(marker_name='my_sky')

You can also remove all the markers.

In [ ]:
viewer.reset_markers()

You can save the active display as PNG file.

In [ ]:
viewer.save('myimage.png')

If you want a second viewer, run the following cell *or* click on the "picture with +" icon on top of the image viewer.

If you click the icon, the viewer name will be auto-generated. Instead of running the next cell, you can run this instead:

    viewer_2_name = 'imviz-1'
    viewer_2 = imviz.app.get_viewer_by_id(viewer_2_name)

In [ ]:
viewer_2_name = 'my-viewer-1'

# Do not pass in viewer_name if you want it auto-generated.
# If auto-generated, viewer name will be 'imviz-1'.
viewer_2 = imviz.create_image_viewer(viewer_name=viewer_2_name)

You can also programmatically control the new viewer, as follows.

In [ ]:
# Show the first image in the second viewer.
imviz.app.add_data_to_viewer(viewer_2_name, "roman_wfi_image_model[DATA]")

In [ ]:
viewer_2.zoom(4)

In [ ]:
viewer_2.cuts = '95%'

In [ ]:
viewer_2.center_on((5, 1))

To destroy this new viewer, run the following cell *or* click the "x" that is right next to the viewer ID.

In [ ]:
imviz.destroy_viewer(viewer_2_name)